<a href="https://colab.research.google.com/github/BrockH3/Intro_to_ml/blob/main/Homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#2
url='/content/drive/My Drive/Intro to ML/datasets/Housing.csv'
housing=pd.DataFrame(pd.read_csv(url))

# set up housing_1 for problem 2 and housing for problem 3
variables =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

#defining map function to change yes to 1 and no to 0
def binary_map(x):
    return x.map({'yes': 1, 'no': 0,})

housing[variables] = housing[variables].apply(binary_map)         #applying it to variable list
housing = housing.drop(columns = 'furnishingstatus')              #housing holds all values in csv file

#scale all inputs to be between 1 and 0
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking','price']

housing[num_vars] = scaler.fit_transform(housing[num_vars])


housing_c = torch.from_numpy(housing.pop('price').to_numpy())
housing_u = torch.from_numpy(housing.to_numpy())


In [ ]:
# randomize training set and validation set for housing set

n = housing_u.shape[0]
n_val = int(0.2 * n)

shuffled = torch.randperm(n)      # randomizes number of samples

train_set = shuffled[:-n_val]     #selects samples to be in training and validation set
val_set = shuffled[-n_val:]

#set training input and output
train_t_u = housing_u[train_set]
train_t_c = housing_c[train_set]
val_t_u = housing_u[val_set]
val_t_c = housing_c[val_set]


In [ ]:
def training_loop(epochs, optimizer, model, loss_function, train_t_u, train_t_c, val_t_u, val_t_c):
  for epoch in range(1, epochs + 1):
      train_t_p = model(train_t_u)
      train_loss = loss_function(train_t_p, train_t_c)

      val_t_p = model(val_t_u)
      val_loss = loss_function(val_t_p, val_t_c)

      optimizer.zero_grad()
      train_loss.backward()
      optimizer.step()

      if (epoch % 500 == 0):
        print('epoch: ', epoch, ' training loss: ', float(train_loss), ' validation loss: ', float(val_loss))

In [ ]:
model = nn.Sequential(nn.Linear(11,32), nn.Tanh(),nn.Linear(32,1))
optimizer = optim.SGD(model.parameters(), lr = 0.001)

training_loop(epochs = 5000,
              optimizer = optimizer,
              model = model.double(),
              loss_function = nn.MSELoss(),
              train_t_u = train_t_u,
              train_t_c = train_t_c,
              val_t_u = val_t_u,
              val_t_c = val_t_c)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch:  500  training loss:  0.02882449634274625  validation loss:  0.031141956927898812
epoch:  1000  training loss:  0.028082332982287746  validation loss:  0.030393849825273482
epoch:  1500  training loss:  0.02759139178074202  validation loss:  0.029912212583294736
epoch:  2000  training loss:  0.02725161414828161  validation loss:  0.02958116397685634
epoch:  2500  training loss:  0.027006077186363043  validation loss:  0.029343159796171428
epoch:  3000  training loss:  0.026821573890826178  validation loss:  0.029164823972197846
epoch:  3500  training loss:  0.026678210408275567  validation loss:  0.02902621389493633
epoch:  4000  training loss:  0.026563702994406503  validation loss:  0.028915085887250093
epoch:  4500  training loss:  0.026470207233875612  validation loss:  0.028823694339449354
epoch:  5000  training loss:  0.026392529958111373  validation loss:  0.02874697759776103


In [ ]:
model = nn.Sequential(nn.Linear(11,32),
                      nn.Tanh(),
                      nn.Linear(32,64),
                      nn.Tanh(),
                      nn.Linear(64,16),
                      nn.Tanh(),
                      nn.Linear(16,1))
optimizer = optim.SGD(model.parameters(), lr = 0.001)

training_loop(epochs = 5000,
              optimizer = optimizer,
              model = model.double(),
              loss_function = nn.MSELoss(),
              train_t_u = train_t_u,
              train_t_c = train_t_c,
              val_t_u = val_t_u,
              val_t_c = val_t_c)


epoch:  500  training loss:  0.027024870705850843  validation loss:  0.030122227378176814
epoch:  1000  training loss:  0.02638650765083517  validation loss:  0.028557364159040942
epoch:  1500  training loss:  0.026345901968859966  validation loss:  0.028469144282942675
epoch:  2000  training loss:  0.026310032086362456  validation loss:  0.028427940254276394
epoch:  2500  training loss:  0.026277178233680115  validation loss:  0.028391890348315393
epoch:  3000  training loss:  0.0262470392124458  validation loss:  0.02835876619586319
epoch:  3500  training loss:  0.026219351147648073  validation loss:  0.028328207305929606
epoch:  4000  training loss:  0.026193879785768297  validation loss:  0.0282999711585475
epoch:  4500  training loss:  0.026170416646924842  validation loss:  0.028273845561430297
epoch:  5000  training loss:  0.02614877575766326  validation loss:  0.028249641289654317


In [4]:
from torchvision import datasets, transforms
datapath = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(datapath, train = True, download = True, transform = transforms.ToTensor())
cifar10val = datasets.CIFAR10(datapath, train = False, download = True, transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(cifar10,batch_size = 64, shuffle = True)
val_loader = torch.utils.data.DataLoader(cifar10val,batch_size = 64, shuffle = True)


100%|██████████| 170498071/170498071 [00:02<00:00, 84753704.79it/s] 


Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


In [ ]:
model = nn.Sequential(nn.Linear(3072,512),
                      nn.Tanh(),
                      nn.Linear(512,10),
                      nn.LogSoftmax(dim=1))

optimizer = optim.SGD(model.parameters(),lr = .01)
loss_function = nn.NLLLoss()

epochs = 300

for epoch in range(epochs):
  for imgs, labels in train_loader:
    batch_size = imgs.shape[0]
    output = model(imgs.view(batch_size,-1))
    loss = loss_function(output,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  if (epoch%50 == 0):
    print('epoch: ', epoch, ' loss: ', float(loss))

epoch:  0  loss:  1.82626211643219
epoch:  50  loss:  1.2656434774398804
epoch:  100  loss:  1.579761028289795
epoch:  150  loss:  0.5713603496551514
epoch:  200  loss:  0.6645681858062744
epoch:  250  loss:  0.39324361085891724


In [8]:
correct = 0
total = 0
with torch.no_grad():
  for img, labels in val_loader:
    batch_size = img.shape[0]
    output = model(img.view(batch_size,-1))
    _, predicted = torch.max(output,dim=1)
    total += labels.shape[0]
    correct+=int((predicted==labels).sum())
print('accuracy: ', correct/total)


accuracy:  0.0921


In [4]:
model = nn.Sequential(nn.Linear(3072,512),
                      nn.Tanh(),
                      nn.Linear(512,256),
                      nn.Tanh(),
                      nn.Linear(256,128),
                      nn.Tanh(),
                      nn.Linear(128,10),
                      nn.LogSoftmax(dim=1))

optimizer = optim.SGD(model.parameters(),lr = .01)
loss_function = nn.NLLLoss()

epochs = 300

for epoch in range(epochs):
  for imgs, labels in train_loader:
    batch_size = imgs.shape[0]
    output = model(imgs.view(batch_size,-1))
    loss = loss_function(output,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  if (epoch%50 == 0):
    print('epoch: ', epoch, ' loss: ', float(loss))

epoch:  0  loss:  1.7726235389709473
epoch:  50  loss:  1.2572051286697388
epoch:  100  loss:  0.9590069651603699
epoch:  150  loss:  0.3668096661567688
epoch:  200  loss:  0.25554153323173523
epoch:  250  loss:  0.015595782548189163


In [6]:
model = nn.Sequential(nn.Linear(3072,512),
                      nn.Tanh(),
                      nn.Linear(512,256),
                      nn.Tanh(),
                      nn.Linear(256,128),
                      nn.Tanh(),
                      nn.Linear(128,10),
                      nn.LogSoftmax(dim=1))


In [13]:
correct = 0
total = 0
with torch.no_grad():
  for img, labels in val_loader:
    batch_size = img.shape[0]
    output = model(img.view(batch_size,-1))
    _, predicted = torch.max(output,dim=1)
    total += labels.shape[0]
    correct+=int((predicted==labels).sum())
print('accuracy: ', correct/total)


accuracy:  0.0921
